In [13]:
#Initialize
from factfinder.calculate import Calculate
from factfinder.special import pivot
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

# pd.options.display.float_format = "{:,.18f}".format

try:
    env_path = "../.env"
    load_dotenv(dotenv_path=env_path)
except:
    print(".env file is missing ...")

calculate = Calculate(api_key=os.environ["API_KEY"], year=2019, source="acs", geography='2010_to_2020')

In [2]:
##INPUTS -- change here
pff_variable = 'mntrvtm'
geotype = 'CT20'
census_geoid_list = ['36047054300']

In [3]:
# See all digits of ratio
ratio = calculate.geo.ratio
ratio.loc[ratio.geoid_ct2020.isin(census_geoid_list), :]


,geoid_ct2010,geoid_ct2020,ratio
877,36047054300,36047054300,0.021407


In [5]:
# Get special base variables
special_base_vars = calculate.meta.get_special_base_variables(pff_variable)


['agttm', 'wrkr16pl', 'cw_wrkdhm', 'wrkrnothm']

In [6]:
# Perfom e and m calculation by calling calculate method
df = calculate.calculate_e_m_special(pff_variable, geotype)
df.head()


,census_geoid,pff_variable,geotype,e,m
0,36005000100,mntrvtm,CT20,NaN,NaN
1,36005000200,mntrvtm,CT20,43.394616,12.466391
2,36005000400,mntrvtm,CT20,52.868440,8.908892
3,36005001600,mntrvtm,CT20,47.987897,8.956524
4,36005001901,mntrvtm,CT20,42.078775,19.419041


In [7]:
# Calculate inputs
inputs_df = calculate.calculate_e_m_multiprocessing(special_base_vars, geotype)
inputs_df.loc[inputs_df.census_geoid.isin(census_geoid_list),:]

,census_geoid,pff_variable,geotype,e,m
854,36047054300,agttm,CT20,150.489297,310.662308
854,36047054300,wrkr16pl,CT20,4.987768,10.560063
854,36047054300,cw_wrkdhm,CT20,0.642202,3.069786
854,36047054300,wrkrnothm,CT20,5.629969,10.997205


In [9]:
def mntrvtm(df: pd.DataFrame, base_variables: list) -> pd.DataFrame:
    df = pivot(df, base_variables)
    df["e"] = df["agttme"] / (df["wrkr16ple"] - df["cw_wrkdhme"])
    df["m"] = (
        1
        / df["wrkrnothme"]
        * np.sqrt(
            df["agttmm"] ** 2
            + (df["agttme"] * df["wrkrnothmm"] / df["wrkrnothme"]) ** 2
        )
    )
    return df

In [14]:
# Apply function to inputs
df = mntrvtm(inputs_df, special_base_vars)
df["pff_variable"] = pff_variable
df["geotype"] = geotype
df.loc[df.census_geoid.isin(census_geoid_list),:]

,census_geoid,agttme,agttmm,wrkr16ple,wrkr16plm,cw_wrkdhme,cw_wrkdhmm,wrkrnothme,wrkrnothmm,e,m,pff_variable,geotype
854,36047054300,150.489297,310.662308,4.987768,10.560063,0.642202,3.069786,5.629969,10.997205,34.630542,75.967139,mntrvtm,CT20


In [7]:
# Peform full calculation (including cleaning/rounding) to show display output
full_calc = calculate(pff_variable, geotype)
full_calc.loc[full_calc.census_geoid.isin(census_geoid_list),:]

,census_geoid,labs_geoid,geotype,labs_geotype,pff_variable,c,e,m,p,z
11,36005002400,2002400,CT20,CT2020,mdfaminc,3.9,112500.0,7295.0,NaN,NaN
